In [ ]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [ ]:
data=pd.read_csv("/content/mission6.csv")
data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data['Diabetes Risk']=data['Diabetes Risk'].map({"Yes":1,"No":0})

In [ ]:
col=["Family History","Smoking Status"]
le={}
for i in col:
  le[i]=LabelEncoder()
  data[i]=le[i].fit_transform(data[i])

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
min=MinMaxScaler()
data[['Age', 'BMI', 'Blood Pressure', 'Physical Activity (hours/week)']]=min.fit_transform(data[['Age', 'BMI', 'Blood Pressure', 'Physical Activity (hours/week)']])

In [ ]:
x=data.drop("Diabetes Risk",axis=1)
y=data["Diabetes Risk"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
lr=LogisticRegression()
lr.fit(x_train,y_train)
pred_lr=lr.predict(x_test)
print(mean_squared_error(y_test,pred_lr))
print(mean_absolute_error(y_test,pred_lr))

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
pred_dt=dt.predict(x_test)
print(mean_squared_error(y_test,pred_dt))
print(mean_absolute_error(y_test,pred_dt))

In [ ]:
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
pred_rf=rf.predict(x_test)
print(mean_squared_error(y_test,pred_rf))
print(mean_absolute_error(y_test,pred_rf))

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x=y_test,y=pred_lr,color='red')
sns.scatterplot(x=y_test,y=pred_dt,color='green')
sns.scatterplot(x=y_test,y=pred_rf,color='yellow')
plt.title("y_test vs y_predict")
plt.xlabel("y_test")
plt.ylabel("pred")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x="Diabetes Risk",y="Age",data=data,color='green')
plt.title("y_test vs y_predict")
plt.xlabel("Diabetes Risk")
plt.ylabel("Age")
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
grid_param = {'max_iter': [30, 40, 50]}
grid_search_model = GridSearchCV(LogisticRegression(),grid_param, cv=5)
grid_search_model.fit(x_train, y_train)
print(grid_search_model.best_params_)
grid_search_model.best_score_

In [ ]:
random_param = {'max_depth': [5,10, 20, 30, 40, 50]}
random_search_model = RandomizedSearchCV(dt,random_param, cv=5)
random_search_model.fit(x_train, y_train)
print(random_search_model.best_params_)
random_search_model.best_score_

In [ ]:
import gradio as gr

In [ ]:
data.columns

In [ ]:
def haneen(age,bmi,BP,PA,FH,SS):
  try:
    input_data=pd.DataFrame({
        "Age":[age],
        "BMI":[bmi],
        "Blood Pressure":[BP],
        "Physical Activity (hours/week)":[PA],
        "Family History":[FH],
        "Smoking Status":[SS]
    })


    for i in ['Family History', 'Smoking Status']:
      input_data[i]=le[i].transform(input_data[i])
    input_data[['Age', 'BMI', 'Blood Pressure', 'Physical Activity (hours/week)']]=min.transform(input_data[['Age', 'BMI', 'Blood Pressure', 'Physical Activity (hours/week)']])
    result=rf.predict(input_data)
    print(result)
    if result[0]==1:
      return "Yes"
    else:
      return "No"
  except Exception as e:
    return str(e)
gr.Interface(
    fn=haneen,
    inputs= [
        gr.Number(label="Age"),
        gr.Number(label="BMI"),
        gr.Number(label="Blood Pressure"),
        gr.Number(label="Physical Activity (hours/week)"),
        gr.Dropdown(choices=["No","Yes"],label="Family History"),
        gr.Dropdown(choices=["No","Yes"],label="Smoking Status")
    ],
     outputs=gr.Textbox(label="prediction")
).launch()